In [2]:
"""We define the class for simulating the ORGaNICs model."""
import os
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pytorch_lightning as pl
from training_scripts.MNIST.model import feedforward
from training_scripts.MNIST.dataset import MnistDataModule
import training_scripts.MNIST.default_config as config
from models.utils.sim_spectrum import sim_solution
import models.ORGaNICs_models as organics
from models.utils.utils import dynm_fun
from torch.func import jacrev, vmap
from matplotlib import colors
import numpy as np
from scipy.optimize import fsolve
from matplotlib.colors import TwoSlopeNorm
from matplotlib.colors import Normalize
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision('medium')

# plt.rc('text', usetex=True)
# plt.rc('font', family='serif')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
cmap = 'viridis'

In [3]:
# We load the model first
# model_name = "MNIST"
model_name = "MNIST_multilayer_layer_120_60"
# model_name = "MNIST_MLP_80"
# model_name = "MNIST_MLP_120_60"
PERMUTED = False
folder_name = "../../tb_logs"
version = 0

In [4]:
def load_model(epoch_number=None, model_name=None, folder_name=None, version=0):
    checkpoint_folder = f'{folder_name}/{model_name}/version_{version}/checkpoints/'
    hparams_path = f'{folder_name}/{model_name}/version_{version}/hparams.yaml'
    checkpoint_files = os.listdir(checkpoint_folder)
    epoch_idx = [int(file.split('epoch=')[1].split('-')[0]) for file in checkpoint_files]

    if epoch_number is not None:
        # If epoch number is provided, find the index of that epoch
        max_idx = epoch_idx.index(epoch_number)
    else:
        # If epoch number is not provided, find the index of the max epoch
        max_idx = epoch_idx.index(max(epoch_idx))

    checkpoint_path = os.path.join(checkpoint_folder, checkpoint_files[max_idx])
    # print(checkpoint_path)
    model = feedforward.load_from_checkpoint(checkpoint_path=checkpoint_path, map_location='cpu', hparams_file=hparams_path)
    model.eval()
    return model

In [5]:
logger = TensorBoardLogger(folder_name, name="Inference")
dm = MnistDataModule(
    data_dir=config.DATA_DIR,
    batch_size=1000,
    num_workers=2,
    permuted=PERMUTED,
)
# calculate training accuracy using pytorch lightning
trainer = pl.Trainer(
    logger=logger,
    max_epochs=1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
epoch_nums = list(range(0, 45))
num_examples = 1000
max_real_eigenvals_training = torch.zeros(len(epoch_nums), num_examples)

for epoch_idx in epoch_nums:
    model = load_model(epoch_number=epoch_idx, model_name=model_name, folder_name=folder_name, version=version)
    trainer.test(model, datamodule=dm)
    model.eval()
    # organics_model = model.org1
    organics_model = model.org2

    N = organics_model.output_size
    tauA = 2.0 + 0 * torch.abs(torch.randn(N) * 0.001)
    tauY = 2.0 + 0 * torch.abs(torch.randn(N) * 0.001)

    # defining parameters from the model
    Wzx = organics_model.Wzx
    Wyy = organics_model.Wr()
    Way = organics_model.Way()
    sigma = organics_model.sigma
    b0 = organics_model.B0()


    # defining the input dependent parameters
    # define x to be the test dataset
    # x = model.activations['org1_input'].clone().to(device)
    x = model.activations['org2_input'].clone().to(device)
    z = F.linear(x, Wzx, bias=None)
    z = z / torch.norm(z, dim=1, keepdim=True)
    b1 = organics_model.B1(x)

    max_real_eigenvals = torch.zeros((num_examples), dtype=torch.float32)
    num_inputs = num_examples

    def _dynamical_fun(vect, z, b1):
        """
        This function defines the dynamics of the ring ORGaNICs model.
        :param x: The state of the network.
        :return: The derivative of the network at the current time-step.
        """
        y = vect[0:N]
        a = vect[N:]
        dydt = (1 / tauY) * (-y + b1 * z
                + (1 - torch.sqrt(torch.relu(a))) * (Wyy @ y))
        dadt = (1 / tauA) * (-a + (sigma * b0) ** 2 + Way @ (torch.relu(a) * y ** 2))
        return torch.cat((dydt, dadt))


    for idx in range(num_inputs):
        z_new = z[idx, :]
        b1_new = b1[idx, :]
        output_y, output_a = organics_model.steady_state(None, b1_new.unsqueeze(0), b0, z_new.unsqueeze(0))
        vect = torch.cat((output_y, output_a), dim=1)
        vect_new = vect.squeeze(0)
        jac = jacrev(_dynamical_fun)(vect_new, z_new, b1_new)
        # jacobians[idx, :, :] = jac
        # eigenvals[idx, :] = torch.linalg.eigvals(jac)
        max_real_eigenvals[idx] = torch.max(torch.real(torch.linalg.eigvals(jac)))
    
    max_real_eigenvals_training[epoch_idx, :] = max_real_eigenvals.detach().clone()



Missing logger folder: ../../tb_logs/Inference
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8040000200271606
         test_f1            0.8040000200271606
        test_loss           1.7255268096923828
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.977899968624115
         test_f1             0.977899968624115
        test_loss           0.1358150988817215
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9799000024795532
         test_f1            0.9799000024795532
        test_loss           0.12889139354228973
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


ValueError: 17 is not in list

In [10]:
torch.save(max_real_eigenvals_training, f'./eigenvals_second_layer_{model_name}_rebuttal.pt')